# Advanced Lane Finding Project

---

**Author:** Sergey Morozov

---

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
# set up logging
import logging
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s [%(levelname)s] %(message)s")

# constants
import numpy as np
CHESSBOARD_IMAGE_DIR = "chessboard_images"
TEST_IMAGE_DIR = "test_images"
OUTPUT_IMAGE_DIR = "output_images"
OUTPUT_VIDEO_DIR = "output_videos"
ABSOLUTE_SOBEL_X=(7, 15, 100)
ABSOLUTE_SOBEL_Y=(7, 15, 100)    # with these values should have no effect on the resulting images
MAGNITUDE_SOBEL=(7, 30, 100)
DIRECTION_SOBEL=(31, 0.5, 1.0)
S_CHANNEL_THRESHOLD=(170, 255)
WARP_SRC = np.float32([(532, 496),
                       (756, 496),
                       (288, 664),
                       (1016, 664)])
WARP_DST = np.float32([(WARP_SRC[2][0], WARP_SRC[2][1] - 298),
                       (WARP_SRC[3][0], WARP_SRC[3][1] - 298),
                       (WARP_SRC[2][0], WARP_SRC[2][1]),
                       (WARP_SRC[3][0], WARP_SRC[3][1])])

# all lane finding code is in advanced_lane_finding.py
from advanced_lane_finding import *

# initialize class instance containing advanced lane line detection methods
lane_finder = AdvancedLaneFinder(
    chessboard_image_dir=CHESSBOARD_IMAGE_DIR,
    absolute_sobel_x=ABSOLUTE_SOBEL_X,
    absolute_sobel_y=ABSOLUTE_SOBEL_Y,
    magnitude_sobel=MAGNITUDE_SOBEL,
    direction_sobel=DIRECTION_SOBEL,
    s_channel_thresh=S_CHANNEL_THRESHOLD,
    warp_perspective=(WARP_SRC, WARP_DST),
)

## Compute Camera Calibration Matrix and Distortion Coefficients

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# get calibration matrix and distortion coefficients
cal_mtx, dist_coeffs, _, _ = lane_finder.calibrate_camera()

# undistort chessboard image calibration2.jpg;
# visially it is the mostly distorted image
test_img = cv2.imread(os.path.join(CHESSBOARD_IMAGE_DIR, "calibration2.jpg"))

# undistort test chessboard image
test_img_undistorted = lane_finder.distortion_correction(image=test_img)

# visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(test_img_undistorted)
ax2.set_title('Undistorted Image', fontsize=30)

# save undistorted image
if not os.path.isdir(OUTPUT_IMAGE_DIR):
    os.mkdir(OUTPUT_IMAGE_DIR)

undist_img_path = os.path.abspath(os.path.join(OUTPUT_IMAGE_DIR, "chessboard_undistorted.jpg"))
ret = cv2.imwrite(undist_img_path, test_img_undistorted)
logging.info("Undistorted chessboard image has been written to %s.", undist_img_path)

Note that in the initial set of chessboard images, provided by Udacity, image *calibration7.jpg* and *calibration15.jpg* had shape (1281, 721) while all other images had shape (1280, 720). These discrepancy led to inaccrate calculations. Images with incorrect shape (*calibration7.jpg* and *calibration15.jpg*) were cropped by 1 pixel and in this repository all images have (1280, 720) shape.

## Apply Distortion Correction to Raw Images

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
%matplotlib inline

# undistort each image in TEST_IMAGE_DIR and save undistorted images to OUTPUT_IMAGE_DIR
test_img_paths = os.listdir(TEST_IMAGE_DIR)

for fname in test_img_paths:
    img = cv2.imread(os.path.join(TEST_IMAGE_DIR, fname))
    img_undist = lane_finder.distortion_correction(image=img)
    
    # save undistorted image
    cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, fname[0:-4] + "_undistorted.jpg"), img_undist)
    
# visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))

img = plt.imread(os.path.join(TEST_IMAGE_DIR, "test1.jpg"))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)

img_undist = plt.imread(os.path.join(OUTPUT_IMAGE_DIR, "test1_undistorted.jpg"))
ax2.imshow(img_undist)
ax2.set_title('Undistorted Image', fontsize=30)

## Create Thresholded Binary Images

In [ ]:
import cv2 
import os
import matplotlib.pyplot as plt
%matplotlib inline

# apply thresholds for each undistorted image in OUTPUT_IMAGE_DIR and save thresholded images to OUTPUT_IMAGE_DIR
output_img_paths = os.listdir(OUTPUT_IMAGE_DIR)

for fname in test_img_paths:
    if fname.startswith('chessboard') or not fname.endswith('_undistorted.jpg'):
        continue
    
    img = cv2.imread(os.path.join(OUTPUT_IMAGE_DIR, fname))
    img_thresh = lane_finder.apply_thresholds(image=img)
    
    # save undistorted image
    cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, fname[0:-4] + "_thresholded.jpg"), img_thresh * 255)

# visualize thresholded images (use difficult cases)
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20,10))

img = plt.imread(os.path.join(OUTPUT_IMAGE_DIR, "test4_undistorted.jpg"))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)

img_thresh = plt.imread(os.path.join(OUTPUT_IMAGE_DIR, "test4_undistorted_thresholded.jpg"))
ax2.imshow(img_thresh, cmap='gray')
ax2.set_title('Thresholded Image', fontsize=30)

img = plt.imread(os.path.join(OUTPUT_IMAGE_DIR, "test5_undistorted.jpg"))
ax3.imshow(img)
ax3.set_title('Original Image', fontsize=30)

img_thresh = plt.imread(os.path.join(OUTPUT_IMAGE_DIR, "test5_undistorted_thresholded.jpg"))
ax4.imshow(img_thresh, cmap='gray')
ax4.set_title('Thresholded Image', fontsize=30)

## Apply Perspective Transform ("Birds-Eye View")

In [ ]:
import cv2 
import os
import matplotlib.pyplot as plt
%matplotlib inline

# apply perspective transform for each undistorted thresholded image in OUTPUT_IMAGE_DIR 
# and save warped images to OUTPUT_IMAGE_DIR
output_img_paths = os.listdir(OUTPUT_IMAGE_DIR)

for fname in output_img_paths:
    if not fname.endswith('_undistorted_thresholded.jpg'):
        continue
    
    img = cv2.imread(os.path.join(OUTPUT_IMAGE_DIR, fname))
    img_warped = lane_finder.warp_perspective(image=img)
    
    # save undistorted image
    cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, fname[0:-4] + "_warped.jpg"), img_warped)

# visualize perspective transformation
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))

img = plt.imread(os.path.join(OUTPUT_IMAGE_DIR, "test3_undistorted_thresholded.jpg"))
ax1.imshow(img, cmap='gray')
ax1.set_title('Original Image', fontsize=30)

img_warped = plt.imread(os.path.join(OUTPUT_IMAGE_DIR, "test3_undistorted_thresholded_warped.jpg"))
ax2.imshow(img_warped, cmap='gray')
ax2.set_title('Warped Image', fontsize=30)

## Detect Lane Pixels and Fit to Find Lane Boundary

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

img = cv2.imread(os.path.join(OUTPUT_IMAGE_DIR, 'test3_undistorted_thresholded.jpg'), 0)

binary_warped = lane_finder.warp_perspective(image=img)


# Assuming you have created a warped binary image called "binary_warped"
# Take a histogram of the bottom half of the image
histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
# Create an output image to draw on and  visualize the result
out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
# Find the peak of the left and right halves of the histogram
# These will be the starting point for the left and right lines
midpoint = np.int(histogram.shape[0]//2)
leftx_base = np.argmax(histogram[:midpoint])
rightx_base = np.argmax(histogram[midpoint:]) + midpoint

# Choose the number of sliding windows
nwindows = 8
# Set height of windows
window_height = np.int(binary_warped.shape[0]//nwindows)
# Identify the x and y positions of all nonzero pixels in the image
nonzero = binary_warped.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
# Current positions to be updated for each window
leftx_current = leftx_base
rightx_current = rightx_base
# Set the width of the windows +/- margin
margin = 100
# Set minimum number of pixels found to recenter window
minpix = 50
# Create empty lists to receive left and right lane pixel indices
left_lane_inds = []
right_lane_inds = []

# Step through the windows one by one
for window in range(nwindows):
    # Identify window boundaries in x and y (and right and left)
    win_y_low = binary_warped.shape[0] - (window+1)*window_height
    win_y_high = binary_warped.shape[0] - window*window_height
    win_xleft_low = leftx_current - margin
    win_xleft_high = leftx_current + margin
    win_xright_low = rightx_current - margin
    win_xright_high = rightx_current + margin
    # Draw the windows on the visualization image
    cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
    cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
    # Identify the nonzero pixels in x and y within the window
    good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
    good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
    # Append these indices to the lists
    left_lane_inds.append(good_left_inds)
    right_lane_inds.append(good_right_inds)
    # If you found > minpix pixels, recenter next window on their mean position
    if len(good_left_inds) > minpix:
        leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
    if len(good_right_inds) > minpix:        
        rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

# Concatenate the arrays of indices
left_lane_inds = np.concatenate(left_lane_inds)
right_lane_inds = np.concatenate(right_lane_inds)

# Extract left and right line pixel positions
leftx = nonzerox[left_lane_inds]
lefty = nonzeroy[left_lane_inds] 
rightx = nonzerox[right_lane_inds]
righty = nonzeroy[right_lane_inds] 

# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)

# Generate x and y values for plotting
ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)